In [ ]:
!pip install datasets

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install bitsandbytes

In [4]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer, util

In [5]:
from huggingface_hub import login

In [6]:
wikipedia_text_corpus = load_dataset('rag-datasets/rag-mini-wikipedia', 'text-corpus')
wikipedia_question_answer = load_dataset('rag-datasets/rag-mini-wikipedia', 'question-answer')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/719 [00:00<?, ?B/s]

data/passages.parquet/part.0.parquet:   0%|          | 0.00/797k [00:00<?, ?B/s]

Generating passages split:   0%|          | 0/3200 [00:00<?, ? examples/s]

data/test.parquet/part.0.parquet:   0%|          | 0.00/54.4k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/918 [00:00<?, ? examples/s]

In [7]:
documents = wikipedia_text_corpus['passages']['passage']

In [8]:
embeddings_model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
document_embeddings = embeddings_model.encode(documents, batch_size=64, show_progress_bar=True)

Batches:   0%|          | 0/50 [00:00<?, ?it/s]

In [10]:
question1 = "Where is Italian spoken around the world?"

In [11]:
question_embedding1 = embeddings_model.encode(question1, batch_size=64, show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
context1 = util.semantic_search(question_embedding1, document_embeddings, top_k=5)

In [13]:
context1

[[{'corpus_id': 1484, 'score': 0.45053398609161377},
  {'corpus_id': 3103, 'score': 0.44615864753723145},
  {'corpus_id': 2185, 'score': 0.4199186861515045},
  {'corpus_id': 996, 'score': 0.3855949342250824},
  {'corpus_id': 44, 'score': 0.3700403869152069}]]

In [15]:
context_doc_ids1 = [c['corpus_id'] for c in context1[0]]

In [16]:
context_doc_ids1

[1484, 3103, 2185, 996, 44]

In [17]:
context_docs1 = [documents[i] for i in context_doc_ids1]

In [19]:
prompt1 = f'Answer the following question using the provided context. Provide the answer using this template: <ANSWER>your answer here</ANSWER>. \n' \
         f'Context:\n' \
         f'Document 1: {context_docs1[0]}\n' \
         f'Document 2: {context_docs1[1]}\n' \
         f'Document 3: {context_docs1[2]}\n\n' \
         f'Document 4: {context_docs1[3]}\n' \
         f'Document 5: {context_docs1[4]}\n\n' \
         f'Question: {question1}'

In [20]:
model_name = 'llama-2-7b-chat-hf'
tokenizer_name = 'llama-2-7b-chat-hf'
padding_size = 1024

In [21]:
tokenizer = AutoTokenizer.from_pretrained(f'meta-llama/{model_name}',
                                          use_auth_token=True,
                                          device_map='cuda:0',
                                          model_max_length=padding_size)
tokenizer.pad_token = tokenizer.bos_token
tokenizer.padding_side = 'left'

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1025: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [22]:
inputs1 = tokenizer(prompt1, truncation=True, padding=True, return_tensors='pt')

In [23]:
input_ids1 = inputs1['input_ids']
attention_masks1 = inputs1['attention_mask']

In [24]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.float16,
                                         bnb_4bit_quant_type='nf4')

In [25]:
model = AutoModelForCausalLM.from_pretrained(f'meta-llama/{model_name}',
                                             device_map='cuda:0',
                                             quantization_config=quantization_config)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [26]:
input_ids1 = input_ids1.to('cuda')

In [27]:
attention_masks1 = attention_masks1.to('cuda')

In [28]:
output_ids1 = model.generate(input_ids=input_ids1,
                            attention_mask=attention_masks1)

In [29]:
generated_answer1 = tokenizer.decode(output_ids1[0],
                                    skip_special_tokens=True,
                                    clean_up_tokenization_spaces=True)

In [30]:
print(generated_answer1)

Answer the following question using the provided context. Provide the answer using this template: <ANSWER>your answer here</ANSWER>. 
Context:
Document 1: Tesla was fluent in many languages. Along with Serbo-Croatian, he also spoke seven other foreign languages: Czech, English, French, German, Hungarian, Italian, and Latin.
Document 2: The official language of Romania is Romanian, an Eastern Romance language related to Italian, French, Spanish, Portuguese and Catalan. Romanian is spoken as a first language by 91% of the population, with Hungarian and Romani being the most important minority languages, spoken by 6.7% and 1.1% of the population, respectively.  Until the 1990s, there was also a substantial number of German-speaking Transylvanian Saxons, even though many have since emigrated to Germany, leaving only 45,000 native German speakers in Romania. In localities where a given ethnic minority makes up more than 20% of the population, that minority's language can be used in the publ

Втор начин

In [31]:
prompt2 = "Where is Italian spoken around the world?"

In [32]:
inputs2 = tokenizer(prompt2, return_tensors="pt").to("cuda")
outputs2 = model.generate(**inputs2, max_length=500)

In [33]:
generated_answer2 = tokenizer.decode(outputs2[0], skip_special_tokens=True)

In [34]:
print(generated_answer2)

Where is Italian spoken around the world?
 nobody is born speaking Italian, but it is a language that is widely spoken and understood around the world. Here are some places where Italian is spoken as a first or second language:

1. Italy: Of course, Italian is the official language of Italy, and it is spoken by the vast majority of the population. It is the language of everyday life, government, education, and media.
2. Switzerland: Italian is one of the official languages of Switzerland, along with German, French, and Romansh. It is spoken in the southern regions of Switzerland, particularly in the canton of Ticino and the province of Verbano-Cusio-Ossola.
3. San Marino: San Marino is a small country located in the Apennine Mountains, and Italian is its official language. It is spoken by the vast majority of the population.
4. Vatican City: Italian is the official language of Vatican City, the sovereign city-state located within Rome, Italy. It is spoken by the Pope and the clergy, as

Трет начин

In [35]:
prompt3 = "Where is Italian spoken around the world? Give me a short answer."

In [36]:
inputs3 = tokenizer(prompt3, return_tensors="pt").to("cuda")
outputs3 = model.generate(**inputs3, max_length=500)

In [37]:
generated_answer3 = tokenizer.decode(outputs3[0], skip_special_tokens=True)

In [38]:
print(generated_answer3)

Where is Italian spoken around the world? Give me a short answer. Hinweis auf die Antwort: Italian is spoken in many countries around the world, but the majority of speakers are located in Italy.
Italian is spoken in many countries around the world, but the majority of speakers are located in Italy. Some other countries where Italian is spoken include:

* San Marino
* Vatican City
* Slovenia (in the region of Friuli-Venezia Giulia)
* Croatia (in the region of Istria and the city of Rijeka)
* Albania (in the northwestern region of Epirus)
* Montenegro (in the coastal region of Lovćen)
* Greece (in the island of Sicily and the region of Calabria)
* Brazil (in the states of São Paulo, Paraná, and Santa Catarina)
* Argentina (in the provinces of Tierra del Fuego, Neuquén, and Mendoza)
* Uruguay (in the departments of Montevideo and Colonia)
* United States (in the states of New York, New Jersey, and Massachusetts)

Italian is also spoken as a second language in many other countries, includ

In [39]:
prompt4 = "When does the 14th season of grey's anatomy come out? Give me a short answer."

In [40]:
inputs4 = tokenizer(prompt4, return_tensors="pt").to("cuda")
outputs4 = model.generate(**inputs4, max_length=500)

In [41]:
generated_answer4 = tokenizer.decode(outputs4[0], skip_special_tokens=True)

In [42]:
print(generated_answer4)

When does the 14th season of grey's anatomy come out? Give me a short answer. obviously, the 14th season of grey's anatomy has not yet been released. The show has not been renewed for a 14th season by ABC. The most recent season, season 13, concluded in April 2020, and there has been no official word on a renewal or release date for season 14.
